# Test performance of NSIDC 0723 COGs

Ian test file

https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1

In [1]:
import os

In [ ]:
env = 'GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies '
env += 'GDAL_MAX_RAW_BLOCK_CACHE_SIZE=200000000 GDAL_SWATH_SIZE=200000000 VSI_CURL_CACHE_SIZE=200000000'
src = '/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1'
dst = os.path.basename(src)+'.tif

### GDALINFO

In [20]:
cmd = f'{env} gdalinfo --debug on {src}'
print(cmd)
!{cmd}

CPL_VSIL_CURL_CACHE_SIZE=300000000 GDAL_SWATH_SIZE=300000000 GDAL_MAX_RAW_BLOCK_CACHE_SIZE=300000000 GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies gdalinfo --debug on /vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1
HTTP: libcurl/7.71.1 OpenSSL/1.1.1h zlib/1.2.11 libssh2/1.9.0 nghttp2/1.41.0
VSICURL: GetFileSize(https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1)=2331426979  response_code=200
VSICURL: Downloading 0-16383 (https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1)...
VSICURL: Got response_code=206
GDAL: GDALOpen(/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1, this=0x7fd415f531d0) succeeds as GTiff.
Driver: GTiff

### GDALWARP

In [14]:
# GDALWARP 3.1 AWS
cmd = f'{env} gdalwarp --debug on -overwrite -t_srs EPSG:4326 -te -54.85 69.31 -52.18 70.26 {src} {dst} 2>gdalwarp.log 1>gdalwarp.out'
print(cmd)
!{cmd}

GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies GDAL_MAX_RAW_BLOCK_CACHE_SIZE=200000000 GDAL_SWATH_SIZE=200000000 VSI_CURL_CACHE_SIZE=200000000 gdalwarp --debug on -overwrite -t_srs EPSG:4326 -te -54.85 69.31 -52.18 70.26 /vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1 GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1.tif 2>gdalwarp.log 1>gdalwarp.out


In [15]:
# Network requests
!grep Downloading gdalwarp.log | wc

       7      28    1183


### GDALTRANSLATE

(getting 1 512x512 tile from a COG should be 2 GET requests (1 for header, 1 for data)

In [16]:
cmd = f'{env} gdal_translate --debug on -srcwin 0 0 512 512 {src} {dst} 2>gdaltranslate.log 1>gdaltranslate.out'
print(cmd)
!{cmd}

GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies GDAL_MAX_RAW_BLOCK_CACHE_SIZE=200000000 GDAL_SWATH_SIZE=200000000 VSI_CURL_CACHE_SIZE=200000000 gdal_translate --debug on -srcwin 0 0 512 512 /vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1 GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1.tif 2>gdaltranslate.log 1>gdaltranslate.out


In [17]:
!grep Downloading gdaltranslate.log | wc

       2       8     326


In [18]:
# try to retrive >10MB woth of pixels from 29520x53220 file
# first 5 row tiles ~288MB *uncompressed**
# http://osgeo-org.1560.x6.nabble.com/gdal-dev-gdalwarp-doesn-t-use-the-VSIL-curl-cache-td5391617.html
# can also use VSI_CACHE_SIZE and VSI_CACHE=FALSE
#GDAL_MAX_RAW_BLOCK_CACHE_SIZE=300000000 (300MB)
# GDAL_MAX_RAW_BLOCK_CACHE_SIZE=100000000 (100MB)
#In addition, a global least-recently-used cache of 16 MB
# CPL_VSIL_CURL_CACHE_SIZE=160000000 #default 16MB

# NOTE thes swath sizes and cache sizes must match **uncompressed** raster size
nrows = 512*5
env = 'CPL_VSIL_CURL_CACHE_SIZE=300000000 GDAL_SWATH_SIZE=300000000 GDAL_MAX_RAW_BLOCK_CACHE_SIZE=300000000 GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies'
dst = 'subset.tif'
cmd = f'{env} gdal_translate --debug on -srcwin 0 0 29520 {nrows} {src} {dst} 2>row.log 1>row.out'
print(cmd)
!{cmd}

CPL_VSIL_CURL_CACHE_SIZE=300000000 GDAL_SWATH_SIZE=300000000 GDAL_MAX_RAW_BLOCK_CACHE_SIZE=300000000 GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies gdal_translate --debug on -srcwin 0 0 29520 2560 /vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1 subset.tif 2>row.log 1>row.out


In [19]:
!grep Downloading row.log | wc

       2       8     326


In [24]:
#%%time
# Time to download on home ethernet
#!curl -O -b ~/.urs_cookies -c ~/.urs_cookies -L -n https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1

# 1.9MB/s --> 

--2020-12-16 11:46:50--  https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1
Resolving n5eil11u.ecs.nsidc.org (n5eil11u.ecs.nsidc.org)... 128.138.97.101
Connecting to n5eil11u.ecs.nsidc.org (n5eil11u.ecs.nsidc.org)|128.138.97.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uat.urs.earthdata.nasa.gov/oauth/authorize?client_id=tiXkwDPzAkY1Xw55KBZeIw&response_type=code&redirect_uri=https%3A%2F%2Fn5eil11u.ecs.nsidc.org%2FTS1_redirect&state=aHR0cDovL241ZWlsMTF1LmVjcy5uc2lkYy5vcmcvVFMxL0RQMC9NRUFTVVJFUy9OU0lEQy0wNzIzLjE5OS8yMDIwLjA3LjI3L0dMX1MxYmtzX21vc2FpY18yN0p1bDIwXzAxQXVnMjBfZ2FtbWEwXzUwbV92MDMuMQ [following]
--2020-12-16 11:46:50--  https://uat.urs.earthdata.nasa.gov/oauth/authorize?client_id=tiXkwDPzAkY1Xw55KBZeIw&response_type=code&redirect_uri=https%3A%2F%2Fn5eil11u.ecs.nsidc.org%2FTS1_redirect&state=aHR0cDovL241ZWlsMTF1LmVjcy5uc2lkYy5vcmcvVFMxL0RQMC9NRUFTVVJFUy9OU0lEQy0w